In [17]:
!pip install -r requirements.txt --user

In [18]:
import time
import boto3

sm = boto3.Session().client("sagemaker")

In [ ]:
def cleanup_project(_project):
    projects = sm.list_projects(NameContains=_project)
    print(f"\n{projects['NextToken']}")
    print("ProjectNames:")
    for project in projects["ProjectSummaryList"]:
        project_name = project["ProjectName"]
        print(f"\n{project_name}")

In [19]:
def get_regions():
    ec2 = boto3.client("ec2")
    regions = ec2.describe_regions()
    return [r["RegionName"] for r in regions["Regions"]]

In [20]:
def cleanup_experiment(_experiment_name):
    trials = sm.list_trials(ExperimentName=_experiment_name)["TrialSummaries"]
    print("TrialNames:")
    for trial in trials:
        trial_name = trial["TrialName"]
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print("\tTrialComponentNames:")
        for component in components_in_trial["TrialComponentSummaries"]:
            component_name = component["TrialComponentName"]
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(0.5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=_experiment_name)
    print(f"\nExperiment {_experiment_name} deleted")

In [21]:
def cleanup_monitoring_schedules(_monitoring_schedule_name):
    sm.delete_monitoring_schedule(MonitoringScheduleName=_monitoring_schedule_name)

In [22]:
def cleanup_endpoints(_endpoint_name):
    sm.delete_endpoint(EndpointName=_endpoint_name)

In [23]:
experiment_name = "Experiment-1"
cleanup_experiment(experiment_name)

sm.list_monitoring_schedules()
monitoring_schedules = sm.list_monitoring_schedules()
for monitoring_schedule in monitoring_schedules["MonitoringScheduleSummaries"]:
    print(monitoring_schedule["MonitoringScheduleName"])
    monitoring_schedule_name = monitoring_schedule["MonitoringScheduleName"]
    cleanup_monitoring_schedules(monitoring_schedule_name)

endpoints = sm.list_endpoints()
for endpoint in endpoints["Endpoints"]:
    print(endpoint["EndpointName"])
    endpoint_name = endpoint["EndpointName"]
    cleanup_endpoints(endpoint_name)

ResourceNotFound: An error occurred (ResourceNotFound) when calling the ListTrials operation: Experiment 'arn:aws:sagemaker:ap-southeast-2:487750889657:experiment/experiment-1' does not exist.

In [24]:
get_regions()

ClientError: An error occurred (UnauthorizedOperation) when calling the DescribeRegions operation: You are not authorized to perform this operation.